In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

In [2]:
start_date = '2022-01-01'
final_date = '2022-01-31'

### SERTI (EXTRAIDO EN EXCEL)

In [3]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\SERTI - COMISIONES\*.csv',recursive=False)
serti_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\SERTI - COMISIONES\\0000000000002610_2021-12-29.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\SERTI - COMISIONES\\0000000000002610_2021-12-30.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\SERTI - COMISIONES\\0000000000002610_2021-12-31.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-01.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-02.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-03.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-04.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-05.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-06.csv',
 'G:\\My Drive\\

In [4]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])                     
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,tarjetahabiente,comision,ivaComision,carrier,phone,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL
0,0,557907,9724,100.00,2021-12-28 18:00:22.455,49679728,Chip,002202,NaN,0,...,ALBERTO RAMIREZ PEREZ,700,112.00,Movistar,******4628,"5,918.00",0000000000002610_2021-12-29.csv,4628,2021-12-28,49679728_4628_9724_100.0
1,1,525678,1991,50.00,2021-12-28 18:01:27.719,49679736,Chip,NaN,NaN,0,...,FLORES CASTANEDA JUAN A,0,0.00,Telcel,******2216,NaN,0000000000002610_2021-12-29.csv,2216,2021-12-28,49679736_2216_1991_50.0
2,2,402766,7023,50.00,2021-12-28 18:02:36.545,49679738,Chip,033735,NaN,0,...,TARJETAHABIENTE,200,32.00,Telcel,******3872,"430,625.00",0000000000002610_2021-12-29.csv,3872,2021-12-28,49679738_3872_7023_50.0
3,3,415231,4828,50.00,2021-12-28 18:03:09.449,49679744,Chip,368181,NaN,0,...,TARJETAHABIENTE,200,32.00,Telcel,******9736,"483,936.00",0000000000002610_2021-12-29.csv,9736,2021-12-28,49679744_9736_4828_50.0
4,4,525678,6792,100.00,2021-12-28 18:03:13.909,49679745,Chip,000621,NaN,0,...,PERFIL EJECUTIVO PRG,400,64.00,Telcel,******9901,"903,006.00",0000000000002610_2021-12-29.csv,9901,2021-12-28,49679745_9901_6792_100.0


In [5]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                 float64
carrier                      object
phone                        object
folio                       float64
file_name                    object
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
dtype: object

In [6]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,tarjetahabiente,comision,ivaComision,carrier,phone,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL


In [7]:
duplicates_serti.shape

(0, 24)

In [8]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [9]:
serti.shape

(364893, 24)

In [10]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                      int64
ivaComision                 float64
carrier                      object
phone                        object
folio                       float64
file_name                    object
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
dtype: object

### DB PRODUCTO

In [11]:
producto_files_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\2022\01. ENERO\DB\TOPUPS ENERO 2022.csv',
                              dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
                                     'status_code':object,'serti_transaction_id':object})
# producto_files_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\11. NOVIEMBRE\DB\TOPUPS NOVIEMBRE 2021.csv',
#                               dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
#                                      'status_code':object})


C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (15,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
# topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'phone_number':object,
#                                             'last4':object,'amount':float,'status_code':object})])
topups_db = pd.concat([producto_files_1])
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['serti_transaction_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['serti_transaction_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,folio_id,receipt_no,last4,authNum,orderId,serti_transaction_id,type,response_code,last4_phone,INDEX_TOTAL
0,3e5aee60-52aa-47d3-a165-c51c38332d5b,562b0db4-9233-4756-9829-af2196e1e445,889bbd4b-5122-4ada-9f2c-c8cccf441c6c,49840348,MOVISTAR,3315507367,2022-01-01,2022-01-01,2022-01-01 00:02:01,2022-01-01 00:02:17,...,"45,331.00",TU-cWGMPfS,7637,059373,46193981,49840348,emv_sale,00,7367,49840348_7367_7637_30.0
1,777ff0ff-b10d-4866-904a-cc487d71b3a2,c1f2cb1e-7546-4433-8fe9-208b5c263f77,2ae8d8ab-9c6e-4983-87e1-f7f4a9dc4bf8,49840351,TELCEL,5548738425,2022-01-01,2022-01-01,2022-01-01 00:02:37,2022-01-01 00:02:42,...,"197,066.00",TU-MW6dO21,9795,023913,46193984,49840351,emv_sale,00,8425,49840351_8425_9795_30.0
2,f9bdcfe7-84fc-4136-a08c-f8d48aaa5ed1,35aea3e2-6b6f-4021-8398-d54798fcc2d9,4bddd411-1d96-405c-a541-4b029da5ee6c,49840356,TELCEL,3320443874,2022-01-01,2022-01-01,2022-01-01 00:03:00,2022-01-01 00:03:06,...,"751,799.00",TU-hGL2HG0,0858,000190,46193989,49840356,emv_sale,00,3874,49840356_3874_0858_30.0
3,0742828e-b5da-4ce4-8aa5-047be3cea226,85677570-691b-4185-a123-5cf41b27989c,28655dac-c3a0-4544-8756-875a9229da32,49840359,ATT,7229091841,2022-01-01,2022-01-01,2022-01-01 00:03:15,2022-01-01 00:03:22,...,"526,056.00",TU-NCJZcBt,3311,011238,46193992,49840359,emv_sale,00,1841,49840359_1841_3311_100.0
4,63ac0e7a-efb3-442d-bf00-5cc0814eb7b2,5f946739-9d80-4d93-8e02-d41f1f86edd2,b412ec01-68ba-40c1-930f-f6db804b633a,49840364,TELCEL,8662118807,2022-01-01,2022-01-01,2022-01-01 00:04:10,2022-01-01 00:04:19,...,"973,241.00",TU-bDvLWpm,5048,319294,46193997,49840364,emv_sale,00,8807,49840364_8807_5048_100.0


In [13]:
duplicates = topups_db[topups_db.duplicated(['serti_transaction_id'], keep=False)]
duplicates.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,folio_id,receipt_no,last4,authNum,orderId,serti_transaction_id,type,response_code,last4_phone,INDEX_TOTAL
83,58420096-598a-47c0-b3bd-480ec337253f,8e992132-a838-46ef-846d-77707a405bc0,35998af8-d842-4725-ace5-00c91f815021,sin_order_id_db,ATT,4775811689,2022-01-01,2022-01-01,2022-01-01 00:47:21,2022-01-01 00:47:22,...,NaN,TU-bz4z6b3,6634,NaN,46194339,sin_order_id_db,emv_sale,H006,1689,sin_order_id_db_1689_6634_200.0
152,cbd1ccc0-9aa8-4daa-a686-ed190d41a198,0eb34fbc-fa32-408b-ada0-e48b6f47c9c1,a3eee8ad-5de3-4af3-97d4-6cacf32c197a,sin_order_id_db,TELCEL,5564323925,2022-01-01,2022-01-01,2022-01-01 01:34:06,2022-01-01 01:34:08,...,NaN,TU-XLudkH1,2022,NaN,46194744,sin_order_id_db,emv_sale,H006,3925,sin_order_id_db_3925_2022_50.0
352,e1e5b729-e026-48fc-af0e-1dd78c356d6b,f9bcd854-9207-4f51-b3bc-7de29b835da0,4910ca4a-8224-4735-9d0b-6e598d0ada61,0,TELCEL,3334759093,2022-01-01,2022-01-01,2022-01-01 08:39:22,2022-01-01 08:39:24,...,NaN,TU-IsSoulp,4920,NaN,46197943,0,emv_sale,91,9093,0_9093_4920_50.0
538,8f25f92d-f5bf-4150-8f91-a9256bc62830,ba634a50-103a-4ebc-9a45-f3cdc94a353f,4b914fa9-8135-4b0b-8c79-8a3433de775d,0,TELCEL,5544708795,2022-01-01,2022-01-01,2022-01-01 10:03:21,2022-01-01 10:03:22,...,NaN,TU-sXUfDjw,2615,NaN,46198610,0,emv_sale,91,8795,0_8795_2615_50.0
592,ecb858a2-b04f-4c01-b980-172223bdf82e,15e641ce-eaba-4cb9-a412-b19dfdbc2627,2c4f6854-56ff-4706-ad5f-3f2266fcccab,0,TELCEL,6367001635,2022-01-01,2022-01-01,2022-01-01 10:16:35,2022-01-01 10:16:37,...,NaN,TU-vv1ARd0,8090,NaN,46198782,0,emv_sale,91,1635,0_1635_8090_50.0


In [14]:
duplicates.shape

(3796, 26)

In [15]:
duplicates['serti_transaction_id'].unique()

array(['sin_order_id_db', '0'], dtype=object)

In [16]:
topups_db.shape

(263773, 26)

In [17]:
topups_db.dtypes

merchant_id                          object
topup_id                             object
transaction_id                       object
clip_external_tracking_id            object
carrier                              object
phone_number                         object
created_at                   datetime64[ns]
updated_at                   datetime64[ns]
created_at_datetime                  object
updated_at_datetime                  object
payment_type                         object
status_code                          object
status_msg                           object
amount                              float64
processor                            object
issuer_auth_code                     object
folio_id                            float64
receipt_no                           object
last4                                object
authNum                              object
orderId                               int64
serti_transaction_id                 object
type                            

### PIVOT DB

In [18]:
pivot_db = pd.pivot_table(topups_db,index=['updated_at']
                            ,columns=['status_code']
                            ,values=['amount','serti_transaction_id']
                            ,aggfunc={'amount':np.sum,
                                     'serti_transaction_id':len},margins=True).reset_index()
pivot_db.fillna(0,inplace=True)
pivot_db.head()

updated_at    amount                            \
status_code                             -3        -5    0          1   
0            2022-01-01 00:00:00 16,830.00 13,070.00 0.00 291,610.00   
1            2022-01-02 00:00:00 25,010.00 16,480.00 0.00 390,870.00   
2            2022-01-03 00:00:00 47,100.00 30,370.00 0.00 600,390.00   
3            2022-01-04 00:00:00 32,320.00 27,200.00 0.00 515,420.00   
4            2022-01-05 00:00:00 43,400.00 21,470.00 0.00 561,700.00   

                       serti_transaction_id                             
status_code        All                   -3     -5    0        1   All  
0           321,510.00               268.00 198.00 0.00 4,508.00  4974  
1           432,360.00               362.00 296.00 0.00 5,824.00  6482  
2           677,860.00               617.00 479.00 0.00 8,859.00  9955  
3           574,940.00               487.00 405.00 0.00 7,859.00  8751  
4           626,570.00               606.00 368.00 0.00 8,492.00  9466

In [19]:
pivot_db.shape

(32, 11)

### DB VS SERTI

In [20]:
cruce_1 = topups_db.merge(serti[['INDEX_TOTAL','transactionId','panTerminacion','last4_phone_serti','serti_date','importe',
                                 'comision','ivaComision','aprobada','carrier']]
                      ,on='INDEX_TOTAL',how='left')
cruce_1['aprobada'].fillna('Not_found',inplace=True)
cruce_1['transactionId'].fillna('Not_found',inplace=True)
cruce_1.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier_x,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,INDEX_TOTAL,transactionId,panTerminacion,last4_phone_serti,serti_date,importe,comision,ivaComision,aprobada,carrier_y
0,3e5aee60-52aa-47d3-a165-c51c38332d5b,562b0db4-9233-4756-9829-af2196e1e445,889bbd4b-5122-4ada-9f2c-c8cccf441c6c,49840348,MOVISTAR,3315507367,2022-01-01,2022-01-01,2022-01-01 00:02:01,2022-01-01 00:02:17,...,49840348_7367_7637_30.0,49840348,7637,7367,2021-12-31,30.00,210.00,33.60,True,Movistar
1,777ff0ff-b10d-4866-904a-cc487d71b3a2,c1f2cb1e-7546-4433-8fe9-208b5c263f77,2ae8d8ab-9c6e-4983-87e1-f7f4a9dc4bf8,49840351,TELCEL,5548738425,2022-01-01,2022-01-01,2022-01-01 00:02:37,2022-01-01 00:02:42,...,49840351_8425_9795_30.0,49840351,9795,8425,2021-12-31,30.00,120.00,19.20,True,Telcel
2,f9bdcfe7-84fc-4136-a08c-f8d48aaa5ed1,35aea3e2-6b6f-4021-8398-d54798fcc2d9,4bddd411-1d96-405c-a541-4b029da5ee6c,49840356,TELCEL,3320443874,2022-01-01,2022-01-01,2022-01-01 00:03:00,2022-01-01 00:03:06,...,49840356_3874_0858_30.0,49840356,0858,3874,2021-12-31,30.00,120.00,19.20,True,Telcel
3,0742828e-b5da-4ce4-8aa5-047be3cea226,85677570-691b-4185-a123-5cf41b27989c,28655dac-c3a0-4544-8756-875a9229da32,49840359,ATT,7229091841,2022-01-01,2022-01-01,2022-01-01 00:03:15,2022-01-01 00:03:22,...,49840359_1841_3311_100.0,49840359,3311,1841,2021-12-31,100.00,700.00,112.00,True,AT&T
4,63ac0e7a-efb3-442d-bf00-5cc0814eb7b2,5f946739-9d80-4d93-8e02-d41f1f86edd2,b412ec01-68ba-40c1-930f-f6db804b633a,49840364,TELCEL,8662118807,2022-01-01,2022-01-01,2022-01-01 00:04:10,2022-01-01 00:04:19,...,49840364_8807_5048_100.0,49840364,5048,8807,2021-12-31,100.00,400.00,64.00,True,Telcel


In [21]:
cruce_1.shape

(263773, 35)

### STATUS 1 ONLY

In [22]:
cruce_1_status_1 = cruce_1[(cruce_1['status_code'] == '1')]
cruce_1_status_1['importe'].fillna(0,inplace=True)
cruce_1_status_1.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier_x,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,INDEX_TOTAL,transactionId,panTerminacion,last4_phone_serti,serti_date,importe,comision,ivaComision,aprobada,carrier_y
0,3e5aee60-52aa-47d3-a165-c51c38332d5b,562b0db4-9233-4756-9829-af2196e1e445,889bbd4b-5122-4ada-9f2c-c8cccf441c6c,49840348,MOVISTAR,3315507367,2022-01-01,2022-01-01,2022-01-01 00:02:01,2022-01-01 00:02:17,...,49840348_7367_7637_30.0,49840348,7637,7367,2021-12-31,30.00,210.00,33.60,True,Movistar
1,777ff0ff-b10d-4866-904a-cc487d71b3a2,c1f2cb1e-7546-4433-8fe9-208b5c263f77,2ae8d8ab-9c6e-4983-87e1-f7f4a9dc4bf8,49840351,TELCEL,5548738425,2022-01-01,2022-01-01,2022-01-01 00:02:37,2022-01-01 00:02:42,...,49840351_8425_9795_30.0,49840351,9795,8425,2021-12-31,30.00,120.00,19.20,True,Telcel
2,f9bdcfe7-84fc-4136-a08c-f8d48aaa5ed1,35aea3e2-6b6f-4021-8398-d54798fcc2d9,4bddd411-1d96-405c-a541-4b029da5ee6c,49840356,TELCEL,3320443874,2022-01-01,2022-01-01,2022-01-01 00:03:00,2022-01-01 00:03:06,...,49840356_3874_0858_30.0,49840356,0858,3874,2021-12-31,30.00,120.00,19.20,True,Telcel
3,0742828e-b5da-4ce4-8aa5-047be3cea226,85677570-691b-4185-a123-5cf41b27989c,28655dac-c3a0-4544-8756-875a9229da32,49840359,ATT,7229091841,2022-01-01,2022-01-01,2022-01-01 00:03:15,2022-01-01 00:03:22,...,49840359_1841_3311_100.0,49840359,3311,1841,2021-12-31,100.00,700.00,112.00,True,AT&T
4,63ac0e7a-efb3-442d-bf00-5cc0814eb7b2,5f946739-9d80-4d93-8e02-d41f1f86edd2,b412ec01-68ba-40c1-930f-f6db804b633a,49840364,TELCEL,8662118807,2022-01-01,2022-01-01,2022-01-01 00:04:10,2022-01-01 00:04:19,...,49840364_8807_5048_100.0,49840364,5048,8807,2021-12-31,100.00,400.00,64.00,True,Telcel


In [23]:
cruce_1_status_1.shape

(237897, 35)

In [24]:
duplicates_cruce_1_status_1 = cruce_1_status_1[cruce_1_status_1.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_status_1.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier_x,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,INDEX_TOTAL,transactionId,panTerminacion,last4_phone_serti,serti_date,importe,comision,ivaComision,aprobada,carrier_y


In [25]:
duplicates_cruce_1_status_1.shape

(0, 35)

In [26]:
cruce_1_final = cruce_1_status_1[(cruce_1_status_1['transactionId'] != 'Not_found')]
cruce_1_final.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier_x,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,INDEX_TOTAL,transactionId,panTerminacion,last4_phone_serti,serti_date,importe,comision,ivaComision,aprobada,carrier_y
0,3e5aee60-52aa-47d3-a165-c51c38332d5b,562b0db4-9233-4756-9829-af2196e1e445,889bbd4b-5122-4ada-9f2c-c8cccf441c6c,49840348,MOVISTAR,3315507367,2022-01-01,2022-01-01,2022-01-01 00:02:01,2022-01-01 00:02:17,...,49840348_7367_7637_30.0,49840348,7637,7367,2021-12-31,30.00,210.00,33.60,True,Movistar
1,777ff0ff-b10d-4866-904a-cc487d71b3a2,c1f2cb1e-7546-4433-8fe9-208b5c263f77,2ae8d8ab-9c6e-4983-87e1-f7f4a9dc4bf8,49840351,TELCEL,5548738425,2022-01-01,2022-01-01,2022-01-01 00:02:37,2022-01-01 00:02:42,...,49840351_8425_9795_30.0,49840351,9795,8425,2021-12-31,30.00,120.00,19.20,True,Telcel
2,f9bdcfe7-84fc-4136-a08c-f8d48aaa5ed1,35aea3e2-6b6f-4021-8398-d54798fcc2d9,4bddd411-1d96-405c-a541-4b029da5ee6c,49840356,TELCEL,3320443874,2022-01-01,2022-01-01,2022-01-01 00:03:00,2022-01-01 00:03:06,...,49840356_3874_0858_30.0,49840356,0858,3874,2021-12-31,30.00,120.00,19.20,True,Telcel
3,0742828e-b5da-4ce4-8aa5-047be3cea226,85677570-691b-4185-a123-5cf41b27989c,28655dac-c3a0-4544-8756-875a9229da32,49840359,ATT,7229091841,2022-01-01,2022-01-01,2022-01-01 00:03:15,2022-01-01 00:03:22,...,49840359_1841_3311_100.0,49840359,3311,1841,2021-12-31,100.00,700.00,112.00,True,AT&T
4,63ac0e7a-efb3-442d-bf00-5cc0814eb7b2,5f946739-9d80-4d93-8e02-d41f1f86edd2,b412ec01-68ba-40c1-930f-f6db804b633a,49840364,TELCEL,8662118807,2022-01-01,2022-01-01,2022-01-01 00:04:10,2022-01-01 00:04:19,...,49840364_8807_5048_100.0,49840364,5048,8807,2021-12-31,100.00,400.00,64.00,True,Telcel


In [27]:
cruce_1_final.shape

(237897, 35)

In [28]:
na_cruce_1_final = cruce_1_status_1[(cruce_1_status_1['transactionId'] == 'Not_found')]
na_cruce_1_final.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier_x,phone_number,created_at,updated_at,created_at_datetime,updated_at_datetime,...,INDEX_TOTAL,transactionId,panTerminacion,last4_phone_serti,serti_date,importe,comision,ivaComision,aprobada,carrier_y


In [29]:
na_cruce_1_final.shape

(0, 35)

In [30]:
cruce_1_status_1.columns

Index(['merchant_id', 'topup_id', 'transaction_id',
       'clip_external_tracking_id', 'carrier_x', 'phone_number', 'created_at',
       'updated_at', 'created_at_datetime', 'updated_at_datetime',
       'payment_type', 'status_code', 'status_msg', 'amount', 'processor',
       'issuer_auth_code', 'folio_id', 'receipt_no', 'last4', 'authNum',
       'orderId', 'serti_transaction_id', 'type', 'response_code',
       'last4_phone', 'INDEX_TOTAL', 'transactionId', 'panTerminacion',
       'last4_phone_serti', 'serti_date', 'importe', 'comision', 'ivaComision',
       'aprobada', 'carrier_y'],
      dtype='object')

### PIVOT STATUS 1 ONLY

In [31]:
pivot_status_1 = pd.pivot_table(cruce_1_status_1,index=['updated_at']
                            ,columns=['aprobada']
                            ,values=['importe','comision','ivaComision','amount','transactionId','serti_transaction_id']
                            ,aggfunc={'importe':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                     'amount':np.sum,
                                     'transactionId':len,
                                     'serti_transaction_id':len},margins=True).reset_index()
pivot_status_1.fillna(0,inplace=True)
pivot_status_1.head()

updated_at     amount                comision               \
aprobada                            True        All         True          All   
0         2022-01-01 00:00:00 291,610.00 291,610.00 1,484,980.00 1,484,980.00   
1         2022-01-02 00:00:00 390,870.00 390,870.00 2,014,050.00 2,014,050.00   
2         2022-01-03 00:00:00 600,390.00 600,390.00 3,090,120.00 3,090,120.00   
3         2022-01-04 00:00:00 515,420.00 515,420.00 2,633,570.00 2,633,570.00   
4         2022-01-05 00:00:00 561,700.00 561,700.00 2,857,600.00 2,857,600.00   

            importe            ivaComision            serti_transaction_id  \
aprobada       True        All        True        All                 True   
0        291,610.00 291,610.00  237,596.80 237,596.80                 4508   
1        390,870.00 390,870.00  322,248.00 322,248.00                 5824   
2        600,390.00 600,390.00  494,419.20 494,419.20                 8859   
3        515,420.00 515,420.00  421,371.20 421,371.20                 7859   
4        561,700.00 561,700.00  457,216.00 457,216.00                 8492   

               transactionId        
aprobada   All          True   All  
0         4508          4508  4508  
1         5824          5824  5824  
2         8859          8859  8859  
3         7859          7859  7859  
4         8492          8492  8492

In [32]:
pivot_status_1.shape

(32, 13)

### PIVOT STATUS 1 POR FECHA Y CARRIER

In [33]:
pivot_carrier = pd.pivot_table(cruce_1_status_1,index=['updated_at','carrier_x']
                            ,columns=['aprobada']
                            ,values=['importe','comision','ivaComision','amount','transactionId','serti_transaction_id']
                            ,aggfunc={'importe':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                     'amount':np.sum,
                                     'transactionId':len,
                                     'serti_transaction_id':len},margins=True).reset_index()
pivot_carrier.fillna(0,inplace=True)
pivot_carrier.head()

updated_at carrier_x     amount              comision  \
aprobada                                      True        All       True   
0         2022-01-01 00:00:00       ATT  39,860.00  39,860.00 279,020.00   
1         2022-01-01 00:00:00  MOVISTAR  36,020.00  36,020.00 252,140.00   
2         2022-01-01 00:00:00    TELCEL 185,430.00 185,430.00 741,720.00   
3         2022-01-01 00:00:00    UNEFON  30,300.00  30,300.00 212,100.00   
4         2022-01-02 00:00:00       ATT  58,430.00  58,430.00 409,010.00   

                       importe            ivaComision             \
aprobada        All       True        All        True        All   
0        279,020.00  39,860.00  39,860.00   44,643.20  44,643.20   
1        252,140.00  36,020.00  36,020.00   40,342.40  40,342.40   
2        741,720.00 185,430.00 185,430.00  118,675.20 118,675.20   
3        212,100.00  30,300.00  30,300.00   33,936.00  33,936.00   
4        409,010.00  58,430.00  58,430.00   65,441.60  65,441.60   

         serti_transaction_id       transactionId        
aprobada                 True   All          True   All  
0                         591   591           591   591  
1                         582   582           582   582  
2                        2713  2713          2713  2713  
3                         622   622           622   622  
4                         841   841           841   841

In [34]:
pivot_carrier.shape

(125, 14)

# EXCEL 

In [35]:
writer = pd.ExcelWriter('CLIP-SERTI TOPUPS ENERO 2022.xlsx', engine='openpyxl',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_db.to_excel(writer, sheet_name='RESUMEN DB')
pivot_status_1.to_excel(writer, sheet_name='RESUMEN')
cruce_1_status_1.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_final.to_excel(writer, sheet_name='FOUND',index=False)
na_cruce_1_final.to_excel(writer, sheet_name='NOT FOUND',index=False)
duplicates_cruce_1_status_1.to_excel(writer, sheet_name='DUPLICADOS_INDEX_TOTAL',index=False)
writer.close()